<h1>Conceito</h1>

<h2><i>Grid search cross validation</i></h2>

A biblioteca *SKLearn* possuí uma classe chamada *grid search cross validation*. Através da mesma, é possível definir um *grid* com diversos valores de diferentes hiperparâmetros a serem testados pela classe.

O que a classe faz portanto é realizar testes de validação cruzada (ou seja, divide os dados passados a ela em diferentes partes de treino e teste) para cada combinação de parâmetros, resultando assim num *score* médio e seus respectivos desvio padrão para cada combinação.

<h2>Validação cruzada aninhada</h2>

Uma prática interessante para, novamente, evitar o *overfitting* é a validação cruzada aninhada. Isso acontece quando são realizados dois ou mais *loops* de validação cruzada, um dentro do outro.

Um exemplo comum de validação cruzada aninhada é **combinar o *grid search cross validation*, que seria o *loop* interno, com uma validação cruzada 'padrão', que seria o *loop* externo**. Neste caso, o que acontece na prática é o seguinte:

1. A validação cruzada padrão (*loop* externo), irá dividir os dados em **dados de treino externos** e **dados de teste externos**, através do método de divisão definido para o *loop* externo
2. Os **dados de teste externos** serão 'guardados' e os **dados de treino externos** serão passados para o *gscv* (*loop* interno)
3. Os **dados de treino externos** serão dividos em **dados de treino internos** e **dados de teste internos**, através do método de divisão definido para o *loop* interno
4. O *gscv* será treinado utilizando os **dados de treino internos** e será calculado o seu *score* utilizando os **dados de teste internos**
5. Os passos 3 e 4 serão repetidos para cada iteração do *loop* interno
6. É criado um modelo então com a combinação de hiperparâmetros com melhor *score* médio considerando todas as iterações. Esse modelo é treinado com todos os **dados de treino externos** e é passado para o parâmetro ***best_estimator_***
7. O *loop* externo então irá calcular o *score* utilizando o **melhor estimador** (definido no passo anterior) e os **dados de teste externos**
8. Os passos 1 a 7 são repetidos para cada iteração do *loop* externo

<h1>Aplicação</h1>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
# Importando a base de dados
dados = pd.read_csv(r'Dados\base.csv')

In [3]:
# Dividindo inputs e outputs
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']].values
y = dados['vendido'].values.ravel()

In [4]:
# Definindo um grid de hiperparâmetros a serem testados

grid_hiperparam = {
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128],
    "min_samples_leaf": [32, 64, 128],
    "criterion": ["gini", "entropy"]
}

In [5]:
# Realizando a busca melhor modelo através da variação dos hiperparâmetros:
SEED = 20
np.random.seed(SEED)

gscv = GridSearchCV(
    DecisionTreeClassifier(),
    grid_hiperparam,
    cv = KFold(n_splits=10, shuffle=True)
)

gscv.fit(x, y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [3, 5],
                         'min_samples_leaf': [32, 64, 128],
                         'min_samples_split': [32, 64, 128]})

In [6]:
# É possível verificar os melhores parâmetros através do seguinte atributo da classe:
gscv.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'min_samples_leaf': 32,
 'min_samples_split': 32}

In [7]:
# É possível verificar todos os resultados através do seguinte atributo da classe:
resultados = pd.DataFrame(gscv.cv_results_)

In [8]:
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021750,0.005179,0.002002,0.000953,gini,3,32,32,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.761,...,0.79,0.786,0.804,0.789,0.777,0.785,0.799,0.7869,0.011113,1
1,0.038951,0.016680,0.006250,0.004993,gini,3,32,64,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.761,...,0.79,0.786,0.804,0.789,0.777,0.785,0.799,0.7869,0.011113,1
2,0.037600,0.022653,0.007950,0.016060,gini,3,32,128,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.761,...,0.79,0.786,0.804,0.789,0.777,0.785,0.799,0.7869,0.011113,1
3,0.025149,0.009664,0.002201,0.001308,gini,3,64,32,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.761,...,0.79,0.786,0.804,0.789,0.777,0.785,0.799,0.7869,0.011113,1
4,0.012899,0.002343,0.001301,0.000641,gini,3,64,64,"{'criterion': 'gini', 'max_depth': 3, 'min_sam...",0.761,...,0.79,0.786,0.804,0.789,0.777,0.785,0.799,0.7869,0.011113,1


In [9]:
# É possível também obter diretamente o melhor estimador através do seguinte atributo:

gscv.best_estimator_

DecisionTreeClassifier(max_depth=3, min_samples_leaf=32, min_samples_split=32)

In [10]:
SEED = 20
np.random.seed(SEED)

cv_score = cross_val_score(gscv, x, y, cv = KFold(n_splits=10, shuffle=True))

In [12]:
# Verificando o score da cross validation para cada iteração:
cv_score

array([0.761, 0.789, 0.789, 0.79 , 0.786, 0.804, 0.789, 0.777, 0.785,
       0.799])

In [16]:
print(f'O score final do modelo, após a otimização de hiperparâmetros e a validação cruzada, está entre o intervalo [{(cv_score.mean() - 2*cv_score.std())*100:.2f}%, {(cv_score.mean() + 2*cv_score.std())*100:.2f}%] e o seu valor médio é {cv_score.mean()*100:.2f}%')

O score final do modelo, após a otimização de hiperparâmetros e a validação cruzada, está entre o intervalo [76.47%, 80.91%] e o seu valor médio é 78.69%
